In [1]:
# %pip install pytorch_lightning

In [2]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import math
import pytorch_lightning as pl
import torchaudio

In [3]:
torch.cuda.is_available()

False

In [4]:
class SpeechEmotionRecognitionModel(pl.LightningModule):
    def __init__(self, input_size, num_classes, dim_model=512, nhead=8, num_encoder_layers=6, num_decoder_layers=6, lr=0.001):
        super(SpeechEmotionRecognitionModel, self).__init__()
        self.lr = lr
        self.transformer = nn.Transformer(d_model=dim_model, nhead=nhead,
                                          num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers)
        self.encoder = nn.Linear(input_size, dim_model)
        self.decoder = nn.Linear(dim_model, num_classes)
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, src):
        src = self.encoder(src)
        output = self.transformer(src, src)
        output = self.decoder(output)
        return output

    def training_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src)
        loss = self.loss_function(output, tgt)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer